# 转置卷积


转置卷积
- 卷积不会增大输入的高宽，通常要么不变、要么减半
- 转置卷积则可以用来增大 输入高宽
$$Y[i:i+h, j:j+w += X[i,j] \cdot K$$

为什么称之为‘转置’
- 对于卷积$Y=X \star W$
  - 可以对$W$构造一个$V$，使得卷积等价于矩阵乘法$Y'=VX'$
  - 这里$Y'$,$X'$是$Y$,$X$对应的向量版本
- 转置卷积则等价于$Y'=V^TX'$
- 如果卷积将输入从$(h,w)$变成了$(h',w')$
  - 同样超参数的转置卷积则从$(h',w')$变成$(h,w)$



**前言**：
- 到目前位置，所见的卷积神经网络层，通常会减少下采样输入图像的空间维度
- 如果输入和输出图像的空间维度相同，在以像素级分类的语义分割中将会很方便
- 为此，在空间维度被卷积神经网络层缩小后，可以使用另一种类型的卷积神经网络层，增加上采样中间层特征图的空间维度

下采样：
- 将高分辨率的数据压缩为低分辨率的过程
- 核心目的：减少数据尺寸、提取核心特征、加速计算

上采样：
- 通过某种方式将低分辨率的信号或图像转换为高分辨率的信号或图像的过程

转置卷积：
- 不是要还原图片，而只是要得到一个pixel level的输出（每个像素的标号）

## 代码

In [1]:
!pip install d2l

In [2]:
import torch
from torch import nn

from d2l import torch as d2l

### 基本操作


**实现基本的转置卷积运算**

In [6]:
def trans_conv(X, K):
  h, w = K.shape
  Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      Y[i: i + h, j: j + w] += X[i, j] * K
  return Y

**验证实现输出**

In [7]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

**使用高级API获得相同的结果**

In [8]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

### 填充、步幅和多通道

**注**：
- 在转置卷积中，填充被应用于**输出**

In [9]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [10]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

对于多个输入和输出通道，转置卷积与常规卷积以相同的方式运作
- 假设输入有$c_i$个通道，且转置卷积为每个通道输入分配了一个$k_h \times k_w$的卷积核张量
- 当指定多个输出通道时，每个输出通道将有一个$c_i \times k_h \times k_w$的卷积核

In [11]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

### 与矩阵变换的联系

In [13]:
X = torch.arange(9.).reshape(3, 3)
K = torch.tensor([[1., 2], [3, 4]])
Y = d2l.corr2d(X, K)  # 类似前面的trans_conv，得到卷积后的结果
Y

tensor([[27., 37.],
        [57., 67.]])

将卷积核`K`重新写为包含大量0的稀疏权重矩阵`W`

In [14]:
def kernel2matrix(K):
  k, W = torch.zeros(5), torch.zeros((4, 9))  # 3*3=9
  k[:2], k[3:5] = K[0, :], K[1, :]
  W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
  return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [15]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [22]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

## 再谈转置卷积

转置卷积
- 转置卷积是一种卷积
  - 将输入和核进行了重新排列
  - 同卷积一般是做下采样不同，通常用作上采样
  - 如果卷积将输入从$(h,w)$变成了$(h',w')$，同样超参数的转置卷积则从$(h',w')$变成$(h,w)$

重新排列输入和核
- 当填充为0步幅为1时
  - 将输入填充$k-1$（$k$是核窗口）
  - 将核矩阵上下、左右翻转
  - 然后做正常卷积（填充0、步幅1）——得到与直接转置卷积一样的结果
- 当填充为$p$步幅为1时
  - 将输入填充$k-p-1$（$k$是核窗口）
  - 将核矩阵上下、左右翻转
  - 然后做正常卷积（填充0、步幅1）——得到与直接转置卷积一样的结果
- 当填充为$p$步幅为$s$时
  - 在行和列之间插入$s-1$行或列
  - 将输入填充$k-p-1$（$k$是核窗口）
  - 将核矩阵上下、左右翻转
  - 然后做正常卷积（填充0、步幅1）——得到与直接转置卷积一样的结果

形状换算
- 输入高（宽）为$n$，核$k$，填充$p$，步幅$s$
- 转置卷积$n'=sn+k-2p-s$
  - 卷积$n' = \left\lfloor \frac{n - k - 2p + s}{s} \right\rfloor \rightarrow n \geq sn' + k - 2p - s$
  - 如果让高矿成倍的增加，则**$k=2p+s$**

同反卷积的关系
- 数学上反卷积（deconvolution）是指卷积的**逆**运算
  - 如果$Y$=conv($X$, $K$)，那么$X$=conv($Y$, $K$)
- 反卷积很少用在深度学习中
  - 这里说的反卷积神经网络指的是用了**转置卷积**的神经网络

总结
- 转置卷积是一种变化了输入和核的卷积，来得到上采样的目的
- 不等同于数学上的反卷积操作


## 练习

1. 卷积输入`X`和转置的卷积输出`Z`具有相同的形状。他们的数值也相同吗？为什么？

In [1]:
!pip install d2l

In [5]:
import torch
from torch import nn
from d2l import torch as d2l

def trans_conv(X, K):
  h, w = K.shape
  Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      Y[i: i + h, j: j + w] = X[i, j] * K
  return Y

X = torch.arange(9.).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Z = trans_conv(Y, K)

Z, X

(tensor([[ 27.,  37.,  74.],
         [ 57.,  67., 134.],
         [171., 201., 268.]]),
 tensor([[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]))

2. 使用矩阵乘法来实现卷积是否有效率？为什么？
- 答
  - 因为卷积操作具有稀疏性和权值共享的特点，而矩阵乘法没有充分利用这些特性